**Team 4 / Preprocessing / D02**

In [278]:
import pandas as pd
import numpy as np
import collections
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import joblib

from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV



In [279]:
df = pd.read_csv ('data/listings.csv')
df.head(5)

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,...,number_of_reviews_l30d,first_review,last_review,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,10595,https://www.airbnb.com/rooms/10595,20211025162728,2021-10-26,"96m2, 3BR, 2BA, Metro, WI-FI etc...",Athens Furnished Apartment No6 is 3-bedroom ap...,Ampelokipi district is nice multinational and ...,https://a0.muscache.com/pictures/f7e19a44-5afe...,37177,https://www.airbnb.com/users/show/37177,...,0,2015-05-25,2019-04-04,00000957568,t,6,6,0,0,0.41
1,10990,https://www.airbnb.com/rooms/10990,20211025162728,2021-10-25,Athens Quality Apartments - Deluxe Apartment,Athens Quality Apartments - Deluxe apartment i...,Ampelokipi district is nice multinational and ...,https://a0.muscache.com/pictures/8645179/c1728...,37177,https://www.airbnb.com/users/show/37177,...,1,2015-11-25,2016-02-22,00001070920,t,6,6,0,0,0.72
2,10993,https://www.airbnb.com/rooms/10993,20211025162728,2021-10-25,Athens Quality Apartments - Studio,The Studio is an <br />-excellent located <br ...,Ampelokipi district is nice multinational and ...,https://a0.muscache.com/pictures/107309527/848...,37177,https://www.airbnb.com/users/show/37177,...,3,2015-10-18,2018-03-31,00000957080,t,6,6,0,0,0.97
3,10995,https://www.airbnb.com/rooms/10995,20211025162728,2021-10-25,"AQA-No2 1-bedroom, smart tv, fiber connection,","AQA No2 is 1-bedroom apartment (47m2), on the ...",Ampelokipi district is nice multinational and ...,https://a0.muscache.com/pictures/6a565613-aaa3...,37177,https://www.airbnb.com/users/show/37177,...,0,2015-12-05,2016-08-06,00000957422,t,6,6,0,0,0.33
4,27262,https://www.airbnb.com/rooms/27262,20211025162728,2021-10-26,"54m2, 1-br, cable tv, wi-fi, metro",Big 1-bedroom apartment that can accommodate 4...,NaN,https://a0.muscache.com/pictures/8651803/4b82b...,37177,https://www.airbnb.com/users/show/37177,...,0,2015-11-12,2017-05-15,00000957579,t,6,6,0,0,0.23


In [280]:
df.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'description',
       'neighborhood_overview', 'picture_url', 'host_id', 'host_url',
       'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'calendar_upd

In [281]:
columns = ['listing_url', 'scrape_id', 'last_scraped', 'name', 'description',
       'neighborhood_overview', 'picture_url', 'host_id', 'host_url',
       'host_name', 'host_location', 'host_about', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count','host_verifications', 'host_has_profile_pic',  'neighbourhood', 'neighbourhood_group_cleansed','bathrooms',
       'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'calendar_updated','has_availability',
       'availability_30', 'availability_60', 'availability_90',
       'calendar_last_scraped','number_of_reviews_ltm', 'number_of_reviews_l30d', 'first_review',
       'last_review','license', 'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms','reviews_per_month','host_since','host_response_time','host_response_rate', 'host_acceptance_rate','host_identity_verified'
       , 'latitude', 'longitude','property_type', 'maximum_nights', 'availability_365','instant_bookable']

In [282]:
df = df.drop(columns, axis='columns')

In [283]:
df.shape

(9582, 12)

In [284]:
df.columns

Index(['id', 'host_is_superhost', 'neighbourhood_cleansed', 'room_type',
       'accommodates', 'bathrooms_text', 'bedrooms', 'beds', 'amenities',
       'price', 'minimum_nights', 'number_of_reviews'],
      dtype='object')

Missing values

In [285]:
print ('Number of missing values in the dataset:',df.isna().sum().sum()) 

Number of missing values in the dataset: 894


In [286]:
print ('Number of rows affected:', sum([True for idx,row in df.iterrows() if any(row.isna())]))

Number of rows affected: 890


In [287]:
print ('Number of collumns affected:',len(df.columns[df.isnull().any()].tolist()))

Number of collumns affected: 4


In [288]:
#How many missing values in each column?

for col in df.columns:
    print (col + ' | number of missing values:', len(df[col][df[col].isna()]))

id | number of missing values: 0
host_is_superhost | number of missing values: 3
neighbourhood_cleansed | number of missing values: 0
room_type | number of missing values: 0
accommodates | number of missing values: 0
bathrooms_text | number of missing values: 10
bedrooms | number of missing values: 813
beds | number of missing values: 68
amenities | number of missing values: 0
price | number of missing values: 0
minimum_nights | number of missing values: 0
number_of_reviews | number of missing values: 0


In [289]:
original = len(df)
df = df.dropna(how='any', subset=['host_is_superhost' ,'bathrooms_text','beds'])
print ('Number of NaN values removed:', original - len(df))

Number of NaN values removed: 81


In [290]:
df.bedrooms.fillna(df.bedrooms.median(), inplace=True)

In [291]:
#Checking for missing values left

df.isna().sum()

id                        0
host_is_superhost         0
neighbourhood_cleansed    0
room_type                 0
accommodates              0
bathrooms_text            0
bedrooms                  0
beds                      0
amenities                 0
price                     0
minimum_nights            0
number_of_reviews         0
dtype: int64

In [292]:
df.dtypes

id                          int64
host_is_superhost          object
neighbourhood_cleansed     object
room_type                  object
accommodates                int64
bathrooms_text             object
bedrooms                  float64
beds                      float64
amenities                  object
price                      object
minimum_nights              int64
number_of_reviews           int64
dtype: object

Encoding categorical features

In [293]:
#neighbourhood_cleansed

df.neighbourhood_cleansed.value_counts()

ΕΜΠΟΡΙΚΟ ΤΡΙΓΩΝΟ-ΠΛΑΚΑ              1841
ΚΟΥΚΑΚΙ-ΜΑΚΡΥΓΙΑΝΝΗ                  829
ΝΕΟΣ ΚΟΣΜΟΣ                          713
ΜΟΥΣΕΙΟ-ΕΞΑΡΧΕΙΑ-ΝΕΑΠΟΛΗ             696
ΑΓΙΟΣ ΚΩΝΣΤΑΝΤΙΝΟΣ-ΠΛΑΤΕΙΑ ΒΑΘΗΣ     595
ΠΑΓΚΡΑΤΙ                             382
ΠΛΑΤΕΙΑ ΑΤΤΙΚΗΣ                      377
ΚΟΛΩΝΑΚΙ                             329
ΑΜΠΕΛΟΚΗΠΟΙ                          328
ΑΚΡΟΠΟΛΗ                             318
ΘΗΣΕΙΟ                               292
ΚΕΡΑΜΕΙΚΟΣ                           244
ΣΤΑΔΙΟ                               223
ΠΕΤΡΑΛΩΝΑ                            210
ΚΥΨΕΛΗ                               201
ΙΛΙΣΙΑ                               169
ΓΟΥΔΙ                                160
ΖΑΠΠΕΙΟ                              140
ΛΥΚΑΒΗΤΤΟΣ                           117
ΠΛΑΤΕΙΑ ΑΜΕΡΙΚΗΣ                     114
ΠΑΤΗΣΙΑ                              113
ΝΙΡΒΑΝΑ                              101
ΓΚΥΖΗ                                 88
ΚΟΛΩΝΟΣ                               82
ΠΕΔΙΟ ΑΡΕΩΣ     

In [294]:
#room_type

df.room_type.value_counts()

Entire home/apt    8429
Private room        865
Hotel room          131
Shared room          76
Name: room_type, dtype: int64

In [295]:
df.room_type.replace({
    'Entire home/apt': 'Home or apt',
    'Private room': 'Private room',
    'Hotel room': 'Hotel room',
    'Shared room': 'Shared room', 
    }, inplace=True)

In [296]:
one_hot = pd.get_dummies(df['room_type'])
df = df.drop('room_type',axis = 1)
df = df.join(one_hot)
df.head(5)

,id,host_is_superhost,neighbourhood_cleansed,accommodates,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,number_of_reviews,Home or apt,Hotel room,Private room,Shared room
0,10595,t,ΑΜΠΕΛΟΚΗΠΟΙ,8,2 baths,3.0,5.0,"[""Kitchen"", ""Free street parking"", ""Crib"", ""Pa...",$79.00,1,32,1,0,0,0
1,10990,t,ΑΜΠΕΛΟΚΗΠΟΙ,4,1 bath,1.0,1.0,"[""Kitchen"", ""Luggage dropoff allowed"", ""Free s...",$50.00,1,52,1,0,0,0
2,10993,t,ΑΜΠΕΛΟΚΗΠΟΙ,2,1 bath,1.0,1.0,"[""Kitchen"", ""Free street parking"", ""Patio or b...",$38.00,1,71,1,0,0,0
3,10995,t,ΑΜΠΕΛΟΚΗΠΟΙ,4,1 bath,1.0,2.0,"[""Kitchen"", ""Free street parking"", ""Patio or b...",$48.00,1,24,1,0,0,0
4,27262,t,ΑΜΠΕΛΟΚΗΠΟΙ,4,1 bath,1.0,1.0,"[""Kitchen"", ""Free street parking"", ""Crib"", ""Pa...",$47.00,1,17,1,0,0,0


In [297]:
#bathrooms_text

df.bathrooms_text.value_counts()

1 bath              6671
1.5 baths            922
2 baths              724
1 private bath       412
1 shared bath        310
2.5 baths            118
3 baths               92
1.5 shared baths      76
2 shared baths        39
3.5 baths             29
3 shared baths        26
4 baths               21
4.5 baths              9
5 baths                9
Half-bath              8
0 baths                6
5 shared baths         5
3.5 shared baths       5
0 shared baths         4
7 baths                2
2.5 shared baths       2
4 shared baths         2
5.5 baths              2
6 baths                2
6.5 baths              2
9.5 baths              1
Shared half-bath       1
12 baths               1
Name: bathrooms_text, dtype: int64

In [298]:
#For later usage

df['bathrooms_text2'] = df['bathrooms_text']
df.head(5)

,id,host_is_superhost,neighbourhood_cleansed,accommodates,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,number_of_reviews,Home or apt,Hotel room,Private room,Shared room,bathrooms_text2
0,10595,t,ΑΜΠΕΛΟΚΗΠΟΙ,8,2 baths,3.0,5.0,"[""Kitchen"", ""Free street parking"", ""Crib"", ""Pa...",$79.00,1,32,1,0,0,0,2 baths
1,10990,t,ΑΜΠΕΛΟΚΗΠΟΙ,4,1 bath,1.0,1.0,"[""Kitchen"", ""Luggage dropoff allowed"", ""Free s...",$50.00,1,52,1,0,0,0,1 bath
2,10993,t,ΑΜΠΕΛΟΚΗΠΟΙ,2,1 bath,1.0,1.0,"[""Kitchen"", ""Free street parking"", ""Patio or b...",$38.00,1,71,1,0,0,0,1 bath
3,10995,t,ΑΜΠΕΛΟΚΗΠΟΙ,4,1 bath,1.0,2.0,"[""Kitchen"", ""Free street parking"", ""Patio or b...",$48.00,1,24,1,0,0,0,1 bath
4,27262,t,ΑΜΠΕΛΟΚΗΠΟΙ,4,1 bath,1.0,1.0,"[""Kitchen"", ""Free street parking"", ""Crib"", ""Pa...",$47.00,1,17,1,0,0,0,1 bath


In [299]:
df.bathrooms_text.replace({
    '1 bath':'Private Bath',
    '1.5 baths':'Private Bath',
    '2 baths':'Private Bath',
    '1 private bath':'Private Bath',
    '1 shared bath':'Shared Bath',
    '2.5 baths':'Private Bath',
    '3 baths':'Private Bath',
    '1.5 shared baths':'Shared Bath',
    '2 shared baths':'Shared Bath',
    '3.5 baths':'Private Bath',
    '3 shared baths':'Shared Bath',
    '4 baths':'Private Bath',
    '4.5 baths':'Private Bath',
    '5 baths':'Private Bath',
    'Half-bath':'Private Bath',
    '0 baths':'Private Bath',
    '5 shared baths':'Shared Bath',
    '3.5 shared baths':'Shared Bath',
    '0 shared baths':'Shared Bath',
    '7 baths':'Private Bath',
    '2.5 shared baths':'Shared Bath',
    '4 shared baths':'Shared Bath',
    '5.5 baths':'Private Bath',
    '6 baths':'Private Bath',
    '6.5 baths':'Private Bath',
    '9.5 baths':'Private Bath',
    'Shared half-bath':'Shared Bath',
    '12 baths':'Private Bath'
    }, inplace=True)

In [300]:
df.bathrooms_text.value_counts()

Private Bath    9031
Shared Bath      470
Name: bathrooms_text, dtype: int64

In [301]:
one_hot = pd.get_dummies(df['bathrooms_text'])
df = df.drop('bathrooms_text',axis = 1)
df = df.join(one_hot)
df.head(5)

,id,host_is_superhost,neighbourhood_cleansed,accommodates,bedrooms,beds,amenities,price,minimum_nights,number_of_reviews,Home or apt,Hotel room,Private room,Shared room,bathrooms_text2,Private Bath,Shared Bath
0,10595,t,ΑΜΠΕΛΟΚΗΠΟΙ,8,3.0,5.0,"[""Kitchen"", ""Free street parking"", ""Crib"", ""Pa...",$79.00,1,32,1,0,0,0,2 baths,1,0
1,10990,t,ΑΜΠΕΛΟΚΗΠΟΙ,4,1.0,1.0,"[""Kitchen"", ""Luggage dropoff allowed"", ""Free s...",$50.00,1,52,1,0,0,0,1 bath,1,0
2,10993,t,ΑΜΠΕΛΟΚΗΠΟΙ,2,1.0,1.0,"[""Kitchen"", ""Free street parking"", ""Patio or b...",$38.00,1,71,1,0,0,0,1 bath,1,0
3,10995,t,ΑΜΠΕΛΟΚΗΠΟΙ,4,1.0,2.0,"[""Kitchen"", ""Free street parking"", ""Patio or b...",$48.00,1,24,1,0,0,0,1 bath,1,0
4,27262,t,ΑΜΠΕΛΟΚΗΠΟΙ,4,1.0,1.0,"[""Kitchen"", ""Free street parking"", ""Crib"", ""Pa...",$47.00,1,17,1,0,0,0,1 bath,1,0


In [302]:
df['bathrooms_text2'] = df['bathrooms_text2'].str.replace('Shared half-bath', '0.5')
df['bathrooms_text2'] = df['bathrooms_text2'].str.replace('Half-bath', '0.5')

df['bathrooms_number'] = (df['bathrooms_text2'].str.split(' ').str[0]).astype(float)

In [303]:
df = df.drop('bathrooms_text2',axis = 1)
df.head(5)

,id,host_is_superhost,neighbourhood_cleansed,accommodates,bedrooms,beds,amenities,price,minimum_nights,number_of_reviews,Home or apt,Hotel room,Private room,Shared room,Private Bath,Shared Bath,bathrooms_number
0,10595,t,ΑΜΠΕΛΟΚΗΠΟΙ,8,3.0,5.0,"[""Kitchen"", ""Free street parking"", ""Crib"", ""Pa...",$79.00,1,32,1,0,0,0,1,0,2.0
1,10990,t,ΑΜΠΕΛΟΚΗΠΟΙ,4,1.0,1.0,"[""Kitchen"", ""Luggage dropoff allowed"", ""Free s...",$50.00,1,52,1,0,0,0,1,0,1.0
2,10993,t,ΑΜΠΕΛΟΚΗΠΟΙ,2,1.0,1.0,"[""Kitchen"", ""Free street parking"", ""Patio or b...",$38.00,1,71,1,0,0,0,1,0,1.0
3,10995,t,ΑΜΠΕΛΟΚΗΠΟΙ,4,1.0,2.0,"[""Kitchen"", ""Free street parking"", ""Patio or b...",$48.00,1,24,1,0,0,0,1,0,1.0
4,27262,t,ΑΜΠΕΛΟΚΗΠΟΙ,4,1.0,1.0,"[""Kitchen"", ""Free street parking"", ""Crib"", ""Pa...",$47.00,1,17,1,0,0,0,1,0,1.0


In [304]:
#price

df['price'] = df['price'].str.replace('$','')
df['price'] = df['price'].str.replace(',','')
df['price'] = pd.to_numeric(df['price'])

C:\Users\melin\AppData\Local\Temp\ipykernel_16872\1035505235.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['price'] = df['price'].str.replace('$','')


In [305]:
df.price.value_counts()

40.0     421
50.0     418
35.0     332
30.0     322
45.0     306
        ... 
990.0      1
219.0      1
283.0      1
264.0      1
625.0      1
Name: price, Length: 357, dtype: int64

In [306]:
#amenities

df["amenities"] = df["amenities"].str.replace('\"', "")
df["amenities"] = df["amenities"].str.replace('[', ",", regex=True)
df["amenities"] = df["amenities"].str.replace(']', "", regex=True)

top_20 = collections.Counter("".join(df["amenities"]).split(",")).most_common(20)

for x in top_20:
    df.loc[df['amenities'].str.contains(x[0]), x[0]] = int(1)

    df[x[0]] = df[x[0]].fillna(0)

df.drop('amenities', axis=1, inplace=True)

In [307]:
df.head(5)

,id,host_is_superhost,neighbourhood_cleansed,accommodates,bedrooms,beds,price,minimum_nights,number_of_reviews,Home or apt,...,Hot water,Dishes and silverware,TV,Cooking basics,Refrigerator,Coffee maker,Dedicated workspace,Bed linens,Washer,Elevator
0,10595,t,ΑΜΠΕΛΟΚΗΠΟΙ,8,3.0,5.0,79.0,1,32,1,...,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
1,10990,t,ΑΜΠΕΛΟΚΗΠΟΙ,4,1.0,1.0,50.0,1,52,1,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
2,10993,t,ΑΜΠΕΛΟΚΗΠΟΙ,2,1.0,1.0,38.0,1,71,1,...,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
3,10995,t,ΑΜΠΕΛΟΚΗΠΟΙ,4,1.0,2.0,48.0,1,24,1,...,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0
4,27262,t,ΑΜΠΕΛΟΚΗΠΟΙ,4,1.0,1.0,47.0,1,17,1,...,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0


In [308]:
 #"Score" of TOP-20 amenities

df['amenities']=df[' Essentials']+df[' Wifi']+df[' Air conditioning']+df[' Long term stays allowed']+df[' Hangers']+df[' Hair dryer']+df[' Iron']+df[' Shampoo']+df['Kitchen']+df[' Heating']+df[' Hot water']+df[' Dishes and silverware']+df[' TV']+df[' Cooking basics']+df[' Refrigerator']+df[' Coffee maker']+df[' Dedicated workspace']+df[' Bed linens']+df[' Washer']+df[' Elevator']

In [309]:
df['amenities']

0       17.0
1       18.0
2       17.0
3       16.0
4       17.0
        ... 
9577    15.0
9578     1.0
9579    19.0
9580    20.0
9581    19.0
Name: amenities, Length: 9501, dtype: float64

In [310]:
df = df.drop([' Essentials', ' Wifi', ' Air conditioning', ' Long term stays allowed',
      ' Hangers', ' Hair dryer', ' Iron', ' Shampoo', 'Kitchen', ' Heating',
       ' Hot water', ' Dishes and silverware', ' TV', ' Cooking basics',
       ' Refrigerator', ' Coffee maker', ' Dedicated workspace', ' Bed linens',
       ' Washer', ' Elevator'], axis=1)

In [311]:

df.to_csv('data/preprocessed_api_data.csv')

df = df.drop(['id'], axis=1)

In [312]:
#host_is_superhost

one_hot = pd.get_dummies(df['host_is_superhost'])
df = df.drop('host_is_superhost',axis = 1)
df = df.join(one_hot)
df.head()

,neighbourhood_cleansed,accommodates,bedrooms,beds,price,minimum_nights,number_of_reviews,Home or apt,Hotel room,Private room,Shared room,Private Bath,Shared Bath,bathrooms_number,amenities,f,t
0,ΑΜΠΕΛΟΚΗΠΟΙ,8,3.0,5.0,79.0,1,32,1,0,0,0,1,0,2.0,17.0,0,1
1,ΑΜΠΕΛΟΚΗΠΟΙ,4,1.0,1.0,50.0,1,52,1,0,0,0,1,0,1.0,18.0,0,1
2,ΑΜΠΕΛΟΚΗΠΟΙ,2,1.0,1.0,38.0,1,71,1,0,0,0,1,0,1.0,17.0,0,1
3,ΑΜΠΕΛΟΚΗΠΟΙ,4,1.0,2.0,48.0,1,24,1,0,0,0,1,0,1.0,16.0,0,1
4,ΑΜΠΕΛΟΚΗΠΟΙ,4,1.0,1.0,47.0,1,17,1,0,0,0,1,0,1.0,17.0,0,1


In [313]:
df.rename(columns={'f': 'not_a_superhost', 't': 'superhost'}, inplace=True)
df.head()

,neighbourhood_cleansed,accommodates,bedrooms,beds,price,minimum_nights,number_of_reviews,Home or apt,Hotel room,Private room,Shared room,Private Bath,Shared Bath,bathrooms_number,amenities,not_a_superhost,superhost
0,ΑΜΠΕΛΟΚΗΠΟΙ,8,3.0,5.0,79.0,1,32,1,0,0,0,1,0,2.0,17.0,0,1
1,ΑΜΠΕΛΟΚΗΠΟΙ,4,1.0,1.0,50.0,1,52,1,0,0,0,1,0,1.0,18.0,0,1
2,ΑΜΠΕΛΟΚΗΠΟΙ,2,1.0,1.0,38.0,1,71,1,0,0,0,1,0,1.0,17.0,0,1
3,ΑΜΠΕΛΟΚΗΠΟΙ,4,1.0,2.0,48.0,1,24,1,0,0,0,1,0,1.0,16.0,0,1
4,ΑΜΠΕΛΟΚΗΠΟΙ,4,1.0,1.0,47.0,1,17,1,0,0,0,1,0,1.0,17.0,0,1


In [314]:
neighbourhood_mapper = df.groupby('neighbourhood_cleansed')['price'].sum()
df['neighbourhood_cleansed'] = df['neighbourhood_cleansed'].map(neighbourhood_mapper)

In [315]:
df.columns

Index(['neighbourhood_cleansed', 'accommodates', 'bedrooms', 'beds', 'price',
       'minimum_nights', 'number_of_reviews', 'Home or apt', 'Hotel room',
       'Private room', 'Shared room', 'Private Bath', 'Shared Bath',
       'bathrooms_number', 'amenities', 'not_a_superhost', 'superhost'],
      dtype='object')

In [316]:
df

,neighbourhood_cleansed,accommodates,bedrooms,beds,price,minimum_nights,number_of_reviews,Home or apt,Hotel room,Private room,Shared room,Private Bath,Shared Bath,bathrooms_number,amenities,not_a_superhost,superhost
0,25392.0,8,3.0,5.0,79.0,1,32,1,0,0,0,1,0,2.0,17.0,0,1
1,25392.0,4,1.0,1.0,50.0,1,52,1,0,0,0,1,0,1.0,18.0,0,1
2,25392.0,2,1.0,1.0,38.0,1,71,1,0,0,0,1,0,1.0,17.0,0,1
3,25392.0,4,1.0,2.0,48.0,1,24,1,0,0,0,1,0,1.0,16.0,0,1
4,25392.0,4,1.0,1.0,47.0,1,17,1,0,0,0,1,0,1.0,17.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9577,165084.0,2,1.0,1.0,10.0,1,0,1,0,0,0,1,0,1.0,15.0,0,1
9578,80939.0,3,1.0,3.0,31.0,1,0,0,0,1,0,0,1,1.0,1.0,1,0
9579,38531.0,13,4.0,9.0,65.0,1,0,1,0,0,0,1,0,2.0,19.0,1,0
9580,48959.0,16,3.0,7.0,79.0,1,0,1,0,0,0,1,0,1.5,20.0,1,0


Feature scaling and outlier handling are implemented in the modelling part in order to improve the models

Saving prepocessed df to be used in D03

In [317]:
df.to_csv('data/preprocessed_data.csv')